In [1]:
import numpy as np
import pandas as pd
import regex as re
import matplotlib.pyplot as plt
import seaborn as sns

pd.set_option('display.max_columns', None)

In [2]:
path_interim = '/home/matteo@COPPET/Documents/data_science/projects/housing_prices_firenze/data/interim/'
path_processed = '/home/matteo@COPPET/Documents/data_science/projects/housing_prices_firenze/data/processed/'

In [3]:
df = pd.read_excel(path_interim+'data.xlsx')
df.head()

,Riferimento e data annuncio,Tipologia,Superficie,Locali,Piano,Totale piani edificio,Tipo proprietà,Altre caratteristiche,Posti auto,Indirizzo,Zona,Prezzo,Informazioni catastali,Spese condominio,Anno di costruzione,Stato,Riscaldamento,Climatizzazione,Efficienza energetica,Prezzo_EUR,Superficie_m2,Riscaldamento_A_C,Tipo_riscaldamento,Alimentazione_riscaldamento,Classe_energetica,Data_annuncio,Ascensore,Accesso_disabili,Piu_livelli,Totale_piani_edificio,Altre_caratteristiche,Fibra ottica,VideoCitofono,Porta blindata,Impianto tv singolo,Parzialmente Arredato,Infissi esterni in vetro / legno,Esposizione doppia,Balcone,Impianto tv centralizzato,Infissi esterni in doppio vetro / PVC,Esposizione esterna,Terrazza,Cancello elettrico,Cantina,Giardino privato,Caminetto,Portiere intera giornata,Infissi esterni in doppio vetro / legno,Armadio a muro,Impianto di allarme,Mansarda,Infissi esterni in doppio vetro / metallo,Solo Cucina Arredata,Esposizione interna,Infissi esterni in vetro / PVC,Giardino comune,Taverna,Impianto tv con parabola satellitare,Arredato,Infissi esterni in triplo vetro / PVC,Infissi esterni in triplo vetro / legno,Infissi esterni in vetro / metallo,Portiere mezza giornata,Cablato,Idromassaggio,Reception,Piscina,Infissi esterni in triplo vetro / metallo,Posti_garage,Posti_esterni,Num_bagni,Superficie_Bins,Num_altri,Num_camere_letto,Num_locali,Num_tot_locali,Prezzo_per_m2
0,A0049 - 05/10/2020,Appartamento,58 m²,"2 (1 camera da letto; 1 altro), 1 bagno, cucin...",1° piano,2 piani,"Intera proprietà, classe immobile media",Fibra ottica\n \n\n ...,NaN,"Firenze, Via del Podestà, Galluzzo",Bellosguardo Galluzzo,€ 260.000,"Classe A/4, rendita € 278",NaN,1900.0,Ottimo / Ristrutturato,"Autonomo, a radiatori, alimentato a metano","Autonomo, freddo/caldo","F 236,08 kWh/m² anno",260000,58.0,Autonomo,radiatori,metano,F,2020-05-10,0,0,0,2.0,"['Fibra ottica', 'VideoCitofono', 'Porta blind...",1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"(0, 60]",1,1,0,2,4482.758621
1,alv275 - 05/10/2020,Appartamento,65 m²,"3 (2 camere da letto; 1 altro), 1 bagno, cucin...","3° piano, con ascensore",4 piani,"Intera proprietà, classe immobile signorile",Fibra ottica\n \n\n ...,NaN,"Firenze, Viale Giovanni Verga, Coverciano, Via...",Coverciano Bellariva,€ 275.000,"Classe A/2, rendita € 888",€ 90/mese,1960.0,Ottimo / Ristrutturato,"Centralizzato, a radiatori, alimentato a metano",Predisposizione impianto,"F ≥ 3,51 kWh/m² anno",275000,65.0,Centralizzato,radiatori,metano,F,2020-05-10,1,0,0,4.0,"['Fibra ottica', 'Porta blindata', 'Balcone', ...",1,0,1,0,0,0,1,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"(60, 80]",1,2,0,3,4230.769231
2,Novoli B - 05/10/2020,Appartamento,54 m²,"2 (1 camera da letto; 1 altro), 1 bagno, cucin...","5° piano, con ascensore",7 piani,"Intera proprietà, classe immobile signorile",Porta blindata\n \n\n ...,NaN,"Firenze, Novoli, Via di Novoli, Via di Novoli, 5",Firenze Nord,€ 195.000,"Classe A/2, rendita € 150",€ 50/mese,1980.0,Ottimo / Ristrutturato,"Centralizzato, a radiatori, alimentato a gas",Predisposizione impianto,G ≥ 175 kWh/m² anno,195000,54.0,Centralizzato,radiatori,gas,G,2020-05-10,1,0,0,7.0,"['Porta blindata', 'Esposizione esterna', 'Ter...",0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,"(0, 60]",1,1,0,2,3611.111111
3,Novoli A - 05/10/2020,Appartamento,56 m²,"2 (1 camera da letto; 1 altro), 1 bagno, cucin...","5° piano, con ascensore",7 piani,"Intera proprietà, classe immobile signorile",Porta blindata\n \n\n ...,NaN,"Firenze, Novoli, Via di Novoli, Via di Novoli, 5",Firenze Nord,€ 195.000,"Classe A/2, rendita € 150",€ 50/mese,1980.0,Ottimo / Ristrutturato,"Centralizzato, a radiatori, alimentato a gas",Predisposizione impianto,G ≥ 175 kWh/m² anno,195000,56.0,Centralizzato,radiatori,gas,G,2020-05-10,1,0,0,7.0,"['Porta blindata', 'Esposizione esterna', 'Ter...",0,0,1,1,0,0,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,

In [ ]:
features = ['Data_annuncio', 'Zona', 'Tipologia', 'Prezzo_EUR', 'Prezzo_per_m2' , 'Superficie_m2',
            'Superficie_Bins', 'Num_tot_locali', 'Num_bagni', 'Stato', 'Tipo_riscaldamento', 'Ascensore']

df = df[features]

### Exploratory Data Analysis

In [ ]:
# Area [m2] distribution - take log to normalize
fig = plt.figure(figsize=(10, 10))
sns.set_style('whitegrid')
sns.distplot(np.log(df['Superficie_m2']), kde=False)

In [ ]:
df['Superficie_m2'].describe()

In [ ]:
# Relationship between price and area - take log to normalize
fig = plt.figure(figsize=(16, 16))
sns.set_style('whitegrid')
sns.scatterplot(x=np.log(df['Superficie_m2']), y=np.log(df['Prezzo_EUR']), hue=df['Zona'])

There is a clear linear relationship between the price and the area where greater area means higher price. It is however difficult to distinguish the various areas in Florence.

In [ ]:
df['Prezzo_EUR'].describe()

In [ ]:
fig = plt.figure(figsize=(10, 10))
sns.set_style('whitegrid')
sns.barplot(y=df['Prezzo_EUR'], x=df['Numero totale di locali'])
plt.xticks(rotation=90)

In [ ]:
fig = plt.figure(figsize=(10, 10))
sns.set_style('whitegrid')
sns.barplot(y=df['Prezzo_EUR'], x=df['Numero di bagni'])

In [ ]:
gb_ordered = (df.loc[df['Zona']!='Non Specificato', :]
              .groupby(['Zona'])
              .agg({'Prezzo_EUR': 'mean'})
              .sort_values('Prezzo_EUR', ascending=False))

fig = plt.figure(figsize=(14, 10))
sns.barplot(gb_ordered['Prezzo_EUR'], y=gb_ordered.index, ci=None)
plt.title('Average price per area')

In [ ]:
gb_ordered = (df.loc[df['Zona']!='Non Specificato', :]
              .groupby(['Superficie_Bins', 'Zona'])
              .agg({'Prezzo_EUR': 'mean'})
              .sort_values('Prezzo_EUR', ascending=False))

fig = plt.figure(figsize=(24, 16))
sns.set_style('whitegrid')
for i, area in enumerate(df['Superficie_Bins'].unique(), 1):
    y_values = gb_ordered.loc[gb_ordered.index.get_level_values('Superficie_Bins').values==area, 'Prezzo_EUR'].reset_index(level=0, drop=True)
    x_values = y_values.index

    plt.subplot(4, 2, i,)
    sns.barplot(y=y_values, x=x_values, ci=None)
    plt.subplots_adjust(left=None, bottom=None, right=None, top=None, wspace=None, hspace=1.5)
    plt.xlabel(xlabel=None)
    plt.title('Area: ' + str(area) + ' m2')
    plt.xticks(rotation=90)

In [ ]:
gb_ordered = (df.loc[df['Zona']!='Non Specificato', :]
              .groupby(['Zona'])
              .agg({'Prezzo_per_m2': 'mean'})
              .sort_values('Prezzo_per_m2', ascending=False))

fig = plt.figure(figsize=(14, 10))
sns.barplot(gb_ordered['Prezzo_per_m2'], y=gb_ordered.index, ci=None)
plt.title('Average price per m2 per area')

In [ ]:
df['Zona'].value_counts()

In [ ]:
sns.pairplot(df)